In [1]:
# pip install rebyte-langchain

In [2]:
from dotenv import load_dotenv

load_dotenv(".env")

True

In [3]:
import os
from rebyte_langchain.rebyte_langchain import RebyteEndpoint

In [4]:
llm = RebyteEndpoint(
    rebyte_api_key=os.getenv("REBYTE_API_KEY"),
    project_id=os.getenv("REBYTE_PROJECT_ID"),
    agent_id=os.getenv("REBYTE_AGENT_ID"),
    session_id="oolLdHU2Rro-Y-HSMtD1z",
    streaming=True,
)

In [5]:
from langchain.schema.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    BaseMessageChunk,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage
)

res=llm.generate(messages=[[HumanMessage("你好")]])
print(res)

{'type': 'run_status', 'content': {'status': 'running', 'run_id': '20c993e2c6d0610982b157dcdd1f391b713f010df41df1d03cbae8b1694c6879'}}
None
{'type': 'block_status', 'content': {'block_type': 'input', 'name': 'INPUT', 'status': 'running', 'success_count': 0, 'error_count': 0}}
None
{'type': 'message', 'content': {'id': 'CsK66VHdWQGQ8YwbR_ovi', 'created_at': 1717731912, 'thread_id': '4618e5c80190ae16cab1:oolLdHU2Rro-Y-HSMtD1z', 'role': 'user', 'content': '你好'}}
None
{'type': 'block_execution', 'content': {'block_type': 'input', 'block_name': 'INPUT', 'execution': [[{'value': {'messages': [{'role': 'user', 'content': '你好'}]}, 'error': None, 'meta': {'thread_id': '4618e5c80190ae16cab1:oolLdHU2Rro-Y-HSMtD1z', 'duration': 6}}]]}}
None
{'type': 'block_status', 'content': {'block_type': 'input', 'name': 'INPUT', 'status': 'succeeded', 'success_count': 1, 'error_count': 0}}
None
{'type': 'block_status', 'content': {'block_type': 'thread', 'name': 'THREAD', 'status': 'running', 'success_count': 

In [ ]:
from langsmith import Client

langsmith_client = Client()

In [ ]:
from tools_basic import tools
from langchain_core.utils.function_calling import convert_to_openai_function
import json

llm_with_tools = llm.bind_tools(tools=tools)
functions = [convert_to_openai_function(t) for t in tools]
print(json.dumps(functions))

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from langchain.agents.format_scratchpad.tools import format_to_tool_messages

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
    )
    | prompt
    | llm_with_tools
    | ToolsAgentOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
streams = executor.astream_events({"input": "今天有什么关于btc的新闻？"}, version="v1")
async for chunk in streams:
    print(chunk)